In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Use Machine Learning and Data Sciences to help explore these ideas:

Q1: What is the best-selling book?

Q2: Visualize order status frequency

Q3: Find a correlation between date and time with order status

Q4: Find a correlation between city and order status

Q5: Find any hidden patterns that are counter-intuitive for a layman

Q6: Can we predict number of orders, or book names in advance?

In [ ]:
#Importing all Necessary libraries For EDA

import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Reading Required Data

df = pd.read_csv("../input/gufhtugu-publications-dataset-challenge/GP Orders - 4.csv",encoding="utf-8", delimiter=',')
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
statistics_of_data = []
for col in df.columns:
  statistics_of_data.append((col,
                             df[col].nunique(),
                             df[col].isnull().sum()*100/df.shape[0],
                             df[col].value_counts(normalize=True, dropna=False).values[0] * 100, 
                             df[col].dtype
                             ))
stats_df = pd.DataFrame(statistics_of_data, columns=['Feature', 'Uniq_val', 'missing_val', 'val_biggest_cat', 'type'])

In [ ]:
stats_df.sort_values('Uniq_val', ascending=False)

In [ ]:
df['Order Status'].unique()

In [ ]:
df['Order Date'].unique()

### Observations::

1. There are 3509 columns in 'Book Name' column,and there are records having more than 1 book ordered as well, We aslo need to seperate them as another record
2. Order Status have 3 unique values:  'Completed', 'Returned', 'Canceled'.
3. We have 99 cities in Pakistan, Which means guftugu delivered books across pakistan as well.
4. Order Date is in DD:MM:YYYY format "'1/20/2021 17:43'".

In [ ]:
## Observing null values

df[df['Book Name'].isna()]

In [ ]:
df[df['City (Billing)'].isna()]

In [ ]:
#Removing these 3 records:::

df.dropna(inplace=True)

In [ ]:
df.head()

#### Q1: What is the best-selling book?

> Lets Find this out

In [ ]:
len(df["Book Name"].unique())

In [ ]:
df.shape

In [ ]:
#Shout out muhammadismail99 for this function

from itertools import chain

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split('/')))

# calculate lengths of splits
lens = df['Book Name'].str.split('/').map(len)

# create new dataframe, repeating or chaining as appropriate
df = pd.DataFrame({'Order Number': np.repeat(df['Order Number'], lens),
                    'Order Status': np.repeat(df['Order Status'], lens),
                    'Book Name': chainer(df['Book Name']),
                    'Order Date': np.repeat(df['Order Date'], lens),
                    'City (Billing)': np.repeat(df['City (Billing)'], lens)})

In [ ]:
df.shape

## Observation:

- Our rows are now increased from 19184 to 33091, As we converted more than 2 orders in record to a seperate record

In [ ]:
## The thing is in Order Status there are cancelled or returned records as well, So for Best selling book it should be the orders which are 
## completed successfully, and for this I selecting only records which have successfull in order status

df1 = df[df['Order Status'] == 'Completed']
df1.head(20)

In [ ]:
df1["Book Name"] = df1["Book Name"].apply(lambda x: x.strip(''))
book_stats = df1["Book Name"].value_counts(ascending=False)
book_stats.head()

In [ ]:
book_stats[book_stats>350]

In [ ]:
import plotly.express as px
top_10_Books_Sold_df = df1["Book Name"].value_counts().nlargest(10).to_frame()

fig = px.bar(top_10_Books_Sold_df, y =top_10_Books_Sold_df['Book Name'], x = top_10_Books_Sold_df.index, title = 'Top 10 most sellers',
            custom_data=[top_10_Books_Sold_df['Book Name'],
                         top_10_Books_Sold_df.index])

fig.update_xaxes(title="Top 10 Books Under Guftugu Publications",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Selling Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',hovertemplate="<br>".join([
                      "Book Name: %{x}",
                      "Count: %{y}",
    ]))
fig.update_layout(hovermode="x unified")
fig.show()

#### Answer1: Best selling Book under Guftugu Publications is " انٹرنیٹ سے پیسہ کمائیں" with 3012 "completed orders"

#### Question 2: Visualize order status frequency

In [ ]:
df['Order Status'].unique()

In [ ]:
df.groupby('Order Status')['Order Status'].agg('count')

In [ ]:
sns.countplot(df['Order Status'])
fig = plt.gcf()
fig.set_size_inches(8,4)
plt.title('Order Status')

#### Observations:::

44 orders got cancelled, While 2396 are returned

### Find a correlation between city and order status

In [ ]:
len(df['City (Billing)'].unique())

In [ ]:
df['City (Billing)'] = df['City (Billing)'].str.lower()

In [ ]:
df['City (Billing)'] = df['City (Billing)'].apply(lambda x: x.strip(''))
city_stats = df['City (Billing)'].value_counts(ascending=False)
city_stats.head()

#### Observations:

1. We need to convert "?? ???", and other these type of values to missing or others.

2. There are different city names like "Karachi", "KARACHI" and "karachi", we need to add them all to one karachi umbrella. This case is with lahore and islamabad too.

3. Lastly, we will set "others" for the cities having very low order rate. 

In [ ]:
def ReplaceString(searchterm,replacedWith,data):
    search_terms = searchterm
    matches = []
    for memo_string in data:#df['City (Billing)']:
        for word in search_terms.split(" "):
            if word not in memo_string:
                break
            else:
                #df['City (Billing)'].replace(memo_string,'karachi',inplace=True)
                data.replace(memo_string,replacedWith,inplace=True)
                matches.append(memo_string) # triggers when the for loop doesn't break

In [ ]:
ReplaceString('karachi','karachi',df['City (Billing)']) # replacing a string having karachi in it to karachi only

In [ ]:
ReplaceString('lahore','lahore',df['City (Billing)']) # replacing a string having lahore in it to lahore only

In [ ]:
ReplaceString('islamabad','islamabad',df['City (Billing)'])

In [ ]:
ReplaceString('rawalpindi','rawalpindi',df['City (Billing)'])

In [ ]:
ReplaceString('faisalabad','faisalabad',df['City (Billing)'])

In [ ]:
ReplaceString('multan','multan',df['City (Billing)'])

In [ ]:
ReplaceString('hyderabad','hyderabad',df['City (Billing)'])

In [ ]:
ReplaceString('????? ??? ??? ???? ??????','unknown',df['City (Billing)'])
ReplaceString('???????','unknown',df['City (Billing)'])
ReplaceString('??? ???','unknown',df['City (Billing)'])
ReplaceString('????????','unknown',df['City (Billing)'])
ReplaceString('?????','unknown',df['City (Billing)'])
ReplaceString('????','unknown',df['City (Billing)'])
ReplaceString('??????','unknown',df['City (Billing)'])
ReplaceString('???????','unknown',df['City (Billing)'])

In [ ]:
#Only those cities which have order status = complete
df1 = df[df['Order Status'] == 'Completed']
df1.head(20)

In [ ]:
# Cities having orders less than 12 are termed as others as they are having very low cardinality to order a book

city_stats_less_than_12 = city_stats[city_stats<=12]

df1['City (Billing)'] =df1['City (Billing)'].apply(lambda x: 'other' if x in city_stats_less_than_12 else x)
len(df1['City (Billing)'].unique())

In [ ]:
df1['City (Billing)'] = df1['City (Billing)'].apply(lambda x: x.strip(''))
city_stats = df1['City (Billing)'].value_counts(ascending=False)
city_stats.head()

In [ ]:
top_10_Books_Sold_By_City_df = df1["City (Billing)"].value_counts().nlargest(10).to_frame()
top_10_Books_Sold_By_City_df

In [ ]:
top_10_Books_Sold_By_City_df = df1["City (Billing)"].value_counts().nlargest(10).to_frame()

top_10_Books_Sold_By_City_df.drop(['other'],axis =0,inplace=True)

fig = px.bar(top_10_Books_Sold_By_City_df, y =top_10_Books_Sold_By_City_df['City (Billing)'],
             x = top_10_Books_Sold_By_City_df.index, title = 'Top 10 Cities Bought Books',
             custom_data=[top_10_Books_Sold_By_City_df['City (Billing)'],
                          top_10_Books_Sold_By_City_df.index])

fig.update_xaxes(title="Top 10 Cities Ordered Books Under Guftugu Publications",title_font=dict(size=18, family='Courier', color='crimson'),
                 linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Selling Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')

fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "City: %{x}",
                      "Count: %{y}",
    ]))





fig.update_layout(hovermode="x unified")

fig.show()

#### Observations:

- Karachi with 5562 orders, tops the list.
- Now Lets start to see the correlation between city and order status.
- Karachi has 386 orders, which are either not complete or not deliverd

In [ ]:
### We have transformed our City data, Now lets start with transforming order status data column
#lets explore order status column

combine = [df]
titlemapping = {'Canceled':0, 'Completed':1,'Returned':2}
for row in combine:
    row["Order Status"] = row["Order Status"].map(titlemapping)
    row['Order Status'] = row['Order Status'].fillna(0)
    row['Order Status'] = row['Order Status'].astype(int)

In [ ]:
df[['City (Billing)','Order Status']].groupby(['Order Status']).agg(['count'])

In [ ]:
# Extract all cities names where order status = Cancelled

df2 = df.loc[(df['Order Status'] == 0)]
df2.head()

In [ ]:
# Extract all cities names where order status = Returned

df3 = df.loc[(df['Order Status'] == 2)]
df3.head()

In [ ]:
df4 = pd.concat([df2, df3])

In [ ]:
df4.shape

In [ ]:
#Cities from where the most number of orders are returned

top_15_cities_returned = df3["City (Billing)"].value_counts().nlargest(15).to_frame()


fig = px.bar(top_15_cities_returned, y =top_15_cities_returned['City (Billing)'], x = top_15_cities_returned.index, title = 'Top 10 Cities From Where Books have been returned',
             custom_data=[top_15_cities_returned['City (Billing)'],
                          top_15_cities_returned.index]
            )

fig.update_xaxes(title="Cities from Books Returned Under Guftugu Publications",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "City: %{x}",
                      "Count: %{y}",
    ]))

fig.update_layout(hovermode="x unified")
fig.show()

In [ ]:
#00
#Cities from where the most number of orders are returned

top_10_cities_cancelled = df2["City (Billing)"].value_counts().nlargest(15).to_frame()

fig = px.bar(top_10_cities_cancelled, y =top_10_cities_cancelled['City (Billing)'], x = top_10_cities_cancelled.index, title = 'Cities Rate for Cancelled Bought Books',
            custom_data=[top_10_cities_cancelled['City (Billing)'],
                          top_10_cities_cancelled.index]
            )

fig.update_xaxes(title="Top 10 Cities cancelled Ordered Books Under Guftugu Publications",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "City: %{x}",
                      "Count: %{y}",
    ]))


fig.update_layout(hovermode="x unified")
fig.show()

#### Observations::

There is no transparent correlation between any city and order returned or cancelled, However it can be concluded as:

1. Karachi, which has highest no. of book buying record, it has too not a good record in cancellation and returning of orders. Karachi has 10 cancelled orders while 376 returned orders.


2. Lahore and Islamabad has also not very good record in terms of returning/ cancelling orders, Although Guftugu publications earned good from these cities.


3. Cities termed as others (Cities which have very low cardinality of buying books) have highest no.'s (562) in returning of orders and 2nd heigst in (7) in cancellation of orders.


4. Which means Guftugu publications can take a moment to think before sending of the books to the cities termed as 'other' categories.

### **Now Lets Start with our new question: "Find a correlation between date and time with order status":**

In [ ]:
## Lets explore date time

df.head()

In [ ]:
# #Shout out to @hussainsaddam12 & @mnavaidd for this codeblock idea
df["Order Date"] = pd.DatetimeIndex(df["Order Date"])
df['date'] = df['Order Date'].dt.date
df['time'] = df['Order Date'].dt.time
df["Day_Name"] = df["Order Date"].dt.day_name()
df["Month_Name"] = df["Order Date"].dt.month_name()
df['year'] = df["Order Date"].dt.year

In [ ]:
df.head()

> We have df2 for Order cancelled: 
> 
> - df2 = df.loc[(df['Order Status'] == 0)]
> Changing its name to df_Order_Cancel
> 
> 
> 
> We have df3 for Order Returned: 
> 
> - df3 = df.loc[(df['Order Status'] == 2)]
> Changing its name to df_Order_Returned
> 
> 
> Now We going to make df_Order_Success For Successful orders**

In [ ]:
df_Order_Cancel = df.loc[(df['Order Status'] == 0)]
df_Order_Cancel.head()

In [ ]:
df_Order_Returned= df.loc[(df['Order Status'] == 2)]
df_Order_Returned.head()

In [ ]:
df_Order_Success = df.loc[(df['Order Status'] == 1)]
df_Order_Success.head()

In [ ]:
df['year'].unique()

In [ ]:
#Order Status == Successfull Orders ....!!!!!!!!!!! W.rt. Year

top_Year_Success_Orders = df_Order_Success["year"].value_counts().to_frame()

fig = px.bar(top_Year_Success_Orders, y =top_Year_Success_Orders['year'], x = top_Year_Success_Orders.index, title = 'Most successful Years For Gufhtugu Publishers',
            custom_data=[top_Year_Success_Orders['year'],
                          top_Year_Success_Orders.index]
            )

fig.update_xaxes(title="Top Earning Years Under Guftugu Publications",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Selling Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "Year: %{x}",
                      "Count: %{y}",
    ]))

fig.update_layout(hovermode="x unified")
fig.show()

### 2020 was the good year for our client, as they sold 23,784 copies despite pandemic. These all are successfull orders (Returned or cancelled are not included)

### In 2021, till last reports they have sold around 6272 copies within less than a month, So lets check it out succes rate of january 2020 and january 2021

In [ ]:
df.head()

In [ ]:
January_2020 = df[(df['Month_Name'] == 'January') & (df['year'] == 2020) & (df['Order Status'] == 1)] 
January_2020.head()


In [ ]:
January_2021 = df[(df['Month_Name'] == 'January') & (df['year'] == 2021) & (df['Order Status'] == 1)] 
January_2021.head()

In [ ]:
MergeJan = pd.concat([January_2020,January_2021])
MergeJan.head()

In [ ]:
#Order Status == Successfull Orders ....!!!!!!!!!!! W.rt. Months

Both_Jans = MergeJan["year"].value_counts().nlargest(2).to_frame()

fig = px.bar(Both_Jans, y =Both_Jans['year'], x = Both_Jans.index, title = 'Comparison of January 2020 & January 2021 Of Gufhtugu Publishers',
            custom_data=[Both_Jans['year'],
                          Both_Jans.index]
            )

fig.update_xaxes(title="January 2020 and 2021",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Selling Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "Year: %{x}",
                      "Count: %{y}",
    ]))
fig.update_layout(hovermode="x unified")
fig.show()

### In 2021 January, they have sold around 6272 copies while in jan'2020; they have sold around 1060 copies only. Their selling rate increased upto 6 times according to stats of january 2021. Remember there are only success orders in the list (Not returned or cancelled)

In [ ]:
#Order Status == Successfull Orders ....!!!!!!!!!!! W.rt. Months in whole data set

top_Month_Success_Orders = df_Order_Success["Month_Name"].value_counts().nlargest(12).to_frame()

fig = px.bar(top_Month_Success_Orders, y =top_Month_Success_Orders['Month_Name'], x = top_Month_Success_Orders.index, title = 'Most successful Months For Gufhtugu Publishers',
            custom_data=[top_Month_Success_Orders['Month_Name'],
                          top_Month_Success_Orders.index]
            )

fig.update_xaxes(title="Top Earning Months Under Guftugu Publications in whole dataset",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Selling Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "Month: %{x}",
                      "Count: %{y}",
    ]))

fig.update_layout(hovermode="x unified")
fig.show()

These are trends for buying books with respect to months: January with (7332) and december with (5331) are top of the lisk while march with (137) and February (96) are at the bottom

In [ ]:
df_Order_Cancel['year'].unique()

In [ ]:
top_Year_cancelled = df_Order_Cancel["year"].value_counts().nlargest(3).to_frame()
fig = px.bar(top_Year_cancelled, y =top_Year_cancelled['year'], x = top_Year_cancelled.index, title = 'Most Orders Cancelled in Each Year',
            custom_data=[top_Year_cancelled['year'],
                          top_Year_cancelled.index]
            )

fig.update_xaxes(title="Top cancelled Ordered Books Under Guftugu Publications w.r.t Year",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')

fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "Year: %{x}",
                      "Count: %{y}",
    ]))
fig.update_layout(hovermode="x unified")

fig.show()


In [ ]:
## Test For No records found in cancellation of record in year 2019
df_test = df[(df['Order Status'] == 0) & (df['year'] == 2019)]
df_test.head()


Hence Proved, No records were found as order cancelled in 2019,and 2020 has only 3 records. Amazing !!!

In [ ]:
top_Month_cancelled = df_Order_Cancel["Month_Name"].value_counts().nlargest(12).to_frame()

fig = px.bar(top_Month_cancelled, y =top_Month_cancelled['Month_Name'], x = top_Month_cancelled.index, title = 'Cancelled Bought Books',
            custom_data=[top_Month_cancelled['Month_Name'],
                          top_Month_cancelled.index]
            )

fig.update_xaxes(title="Cancelled Ordered Books w.r.t. Months Under Guftugu Publications",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "Month: %{x}",
                      "Count: %{y}",
    ]))

fig.update_layout(hovermode="x unified")
fig.show()


In [ ]:
# Returning of books

top_Month_Returned = df_Order_Returned["Month_Name"].value_counts().nlargest(12).to_frame()

fig = px.bar(top_Month_Returned, y =top_Month_Returned['Month_Name'], x = top_Month_Returned.index, title = 'Rate for Returning Bought Books',
            custom_data=[top_Month_Returned['Month_Name'],
                          top_Month_Returned.index]
            )

fig.update_xaxes(title="Most Returned Books w.r.t Months Under Guftugu Publications",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "Month: %{x}",
                      "Count: %{y}",
    ]))





fig.update_layout(hovermode="x unified")
fig.show()

In [ ]:

top_Year_returned = df_Order_Returned["year"].value_counts().nlargest(3).to_frame()

#top_10_cities_cancelled = df2["City (Billing)"].value_counts().nlargest(15).to_frame()

#fig = px.bar(top_10_cities_cancelled, y =top_10_cities_cancelled['City (Billing)'], x = top_10_cities_cancelled.index, title = 'Cities Rate for Cancelled Bought Books')
fig = px.bar(top_Year_returned, y =top_Year_returned['year'], x = top_Year_returned.index, title = 'Most Orders Returned in Each Year',
            custom_data=[top_Year_returned['year'],
                          top_Year_returned.index])

fig.update_xaxes(title="Top returned Ordered Books Under Guftugu Publications w.r.t Year",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "Year: %{x}",
                      "Count: %{y}",
    ]))





fig.update_layout(hovermode="x unified")
fig.show()

### Observations::
There is again no transparent correlation between any time of year and order returned or cancelled, However it can be concluded as:

1. 2021 is So far so good for guftugu publications, There are around 6272 successful orders and 3 cancelled while 477 returned books. 


2. 2020 was pretty good for guftugu publications, despite of pandemic they have sold around 23,784 copies with "انٹرنیٹ سے پیسہ کمائیں" was the best seller.

3. Most cancelled orders are in august while Feb, march, april and june has no cancelled orders in whole dataset.

4. Most returning orders are in 2020, with  count = 1882; Most in December and January

### Hidden Patterns that are Counter-Intuitive For a Lay man

There are some questions which I have created, and I think it would be fascinating to know:

1. Which day is suitable for marketing of books on social media ?


2. Which book is being returned/ cancelled mostly ?


3. Which book is best best seller for top book buying Cities ?

In [ ]:
## Starting with Question#1, We need to know that which day is suitable for marketing? I will discover the pattern of which day the books sold the most.

df.head()

In [ ]:
## All successfull orders::

df_Order_Success.head()

In [ ]:
# Most successful Day

top_Day_Success = df_Order_Success["Day_Name"].value_counts().nlargest(7).to_frame()

fig = px.bar(top_Day_Success, y =top_Day_Success['Day_Name'], x = top_Day_Success.index, title = 'Rate for Successful orders w.r.t Days',
            custom_data=[top_Day_Success['Day_Name'],
                          top_Day_Success.index]
            )

fig.update_xaxes(title="Most Books Bought w.r.t Day Under Guftugu Publications",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "Day: %{x}",
                      "Count: %{y}",
    ]))





fig.update_layout(hovermode="x unified")
fig.show()

### Which day is suitable for marketing of books on social media ?

### Answer is "Saturday" & "Sunday" with more than 6000 and close to 6000 books respectively have been sold in both of the days.

In [ ]:
## Question #02: Which book is being returned/ cancelled mostly ?
df_Order_Returned.head()

In [ ]:
df_Order_Cancel.head()

In [ ]:
df_Ordered_Cnacelled_Merge = pd.concat([df_Order_Returned,df_Order_Cancel])
df_Ordered_Cnacelled_Merge.head()

In [ ]:
top_Book_Returned_Can = df_Ordered_Cnacelled_Merge["Book Name"].value_counts().nlargest(10).to_frame()

fig = px.bar(top_Book_Returned_Can, y =top_Book_Returned_Can['Book Name'], x = top_Book_Returned_Can.index, title = 'Rate for Un-Successful orders w.r.t Books',
            custom_data=[top_Book_Returned_Can['Book Name'],
                          top_Book_Returned_Can.index]
            )

fig.update_xaxes(title="Most Books Returned Or Cancelled Under Guftugu Publications",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "Book: %{x}",
                      "Count: %{y}",
    ]))





fig.update_layout(hovermode="x unified")
fig.show()

### Which book is being returned/ cancelled mostly ?
### Answer is "انٹرنیٹ سے پیسہ کمائیں" has the most returned/cancelled rate with count = 485.

In [ ]:
#### Question#3: Which book is best best seller for top book buying Cities ?

#All successful orders:

df_Order_Success.head()


In [ ]:
df_Order_Success_In_Top2Cities = df_Order_Success[(df_Order_Success['City (Billing)'] == 'karachi' )| 
                                                                    (df_Order_Success['City (Billing)'] == 'lahore')]
df_Order_Success_In_Top2Cities.head()

In [ ]:
top_Book_In_Top_Cities = df_Order_Success_In_Top2Cities["Book Name"].value_counts().nlargest(10).to_frame()

fig = px.bar(top_Book_In_Top_Cities, y =top_Book_In_Top_Cities['Book Name'], x = top_Book_In_Top_Cities.index, title = 'Rate for Successful Books orders w.r.t Top 2 Cities',
            custom_data=[top_Book_In_Top_Cities['Book Name'],
                         #top_Book_In_Top_Cities['City (Billing)']
                          top_Book_In_Top_Cities.index])


fig.update_xaxes(title="Most Books Ordered in respective City Under Guftugu Publications",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_yaxes(title="Books Count",title_font=dict(size=18, family='Courier', color='crimson'), linecolor='black', mirror=True,gridcolor='red')
fig.update_traces(marker_color='purple',
                  hovertemplate="<br>".join([
                      "Book: %{x}",
                      "City: %{y}",
    ]))





fig.update_layout(hovermode="x unified")
fig.show()

### Answer is: انٹرنیٹ سے پیسہ کمائیں is the best seller in top 2 cities. i.e. Karachi and lahore

### Thats all for EDA of Gufhtugu Data set, Hope you have enjoyed whole notebook. I will cover prediction of number of orders or book name in my new notebook very soon Insha-ALLAH. Till then upvote this notebook if you liked it. Thankyou.*******